In [1]:
%pip install opencv-python
%pip install ultralytics
%pip install torch
%pip install uuid

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/77/df/b56175c3fb5bc058774bdcf35f5a71cf9c3c5b909f98a1c688eb71cd3b1f/opencv_python-4.9.0.80-cp37-abi3-macosx_11_0_arm64.whl.metadata
  Using cached opencv_python-4.9.0.80-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
  Obtaining dependency information for numpy>=1.21.2 from https://files.pythonhosted.org/packages/20/f7/b24208eba89f9d1b58c1668bc6c8c4fd472b20c45573cb767f59d49fb0f6/numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl.metadata
  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached opencv_python-4.9.0.80-cp37-abi3-macosx_11_0_arm64.whl (35.4 MB)
Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for ultral

In [18]:
import os
os.environ["OPENCV_VIDEOIO_MSMF_ENABLE_HW_TRANSFORMS"] = "0"
import cv2
from ultralytics import YOLO  
import numpy as np
import torch
import uuid

# Set up environment and device
resize_width = 640
resize_height = 480
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Paths and file settings
weight_path = r'weights/yolov9-c.pt'  # Path to YOLOv9 weights
path = r'Experimenter_9110002_53.mp4'
fileid = str(uuid.uuid4())
record_path = f'yolov9_object_{fileid}.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')

# Create model
model=YOLO(weight_path)
camera=cv2.VideoCapture(path)
width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
fps=int(camera.get(cv2.CAP_PROP_FPS))
height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(camera.get(cv2.CAP_PROP_FRAME_COUNT))

frame_y= height//2    # Frame shape values and precalculation of the mask
frame_x= width//2
n_channels=3
frame_shape=np.array([frame_x,frame_y])   #To get the frame shape
video_write=cv2.VideoWriter(record_path, fourcc,fps, (resize_width,resize_height))

while camera.isOpened():
    # try:
        ret,frame=camera.read()
        if not ret:
            print("End of Video")
            break
        frame_scene=frame[:height//2,:width//2]
        frame_show=frame_scene.copy()
        results=model.predict(frame_scene)
        result=results[0]
        break
        if cv2.waitKey(1) & 0xFF==27:
            print("Quitting the program")
            break
    # except Exception as e:
    #     print(e)
camera.release()
video_write.release()
cv2.destroyAllWindows()

TypeError: ERROR ❌️ weights/yolov9-c.pt appears to be an Ultralytics YOLOv5 model originally trained with https://github.com/ultralytics/yolov5.
This model is NOT forwards compatible with YOLOv8 at https://github.com/ultralytics/ultralytics.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'

In [9]:
result[0].boxes.xyxy.numpy().astype(int)

array([[471, 277, 490, 292]])

In [11]:
result[0].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([2.])
conf: tensor([0.3522])
data: tensor([[4.7113e+02, 2.7732e+02, 4.9006e+02, 2.9204e+02, 3.5217e-01, 2.0000e+00]])
id: None
is_track: False
orig_shape: (540, 960)
shape: torch.Size([1, 6])
xywh: tensor([[480.5947, 284.6831,  18.9311,  14.7194]])
xywhn: tensor([[0.5006, 0.5272, 0.0197, 0.0273]])
xyxy: tensor([[471.1292, 277.3234, 490.0603, 292.0428]])
xyxyn: tensor([[0.4908, 0.5136, 0.5105, 0.5408]])

In [16]:
for box in result[0].boxes:
    box.xyxy.numpy().astype(int)[0]
    x0,y0,x1,y1=box.xyxy.numpy().astype('int')[0]
    frame_show = cv2.rectangle(frame_show, [x0,y0] , [x1,y1], (255,0,0), 2) # blue color
    cv2.imshow('test',frame_show)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
for box in result.boxes:
            cords = box.xyxy[0].tolist()
            cords = [round(x) for x in cords]
            conf = round(box.conf[0].item(), 2)
            start_point = np.array((cords[0],cords[1]))/letterbox_shape   # First x then y (Normalising the points)
            start_point=start_point*frame_shape
            start_point=start_point.astype(int,copy=False)
            end_point = np.array((cords[2], cords[3]))/letterbox_shape
            end_point=end_point*frame_shape     # Getting the points into the final frame shape
            end_point=end_point.astype(int,copy=False)
            if conf > 0.7:  # This value needs to be played with
                frame_show = cv2.rectangle(frame_show, start_point , end_point, (255,0,0), 2) # blue color

        frame_show=cv2.resize(frame_show,(resize_width,resize_height))
        cv2.imshow('frame_scene',frame_show)
        video_write.write(frame_show)

In [17]:
cv2.destroyAllWindows()